In [1]:
%load_ext autoreload
%autoreload 1
%aimport utils.utils
    
import argparse
import os
import json

import torch
import openai
from tqdm import tqdm

from llm.chatgpt import init_chatgpt, ask_llm
from utils.enums import LLM
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer

from utils.post_process import process_duplication, get_sqls
from utils.utils import get_template, get_answer

In [2]:
QUESTION_FILE = "questions.json"
class args:
    model = "granite-8b-instruct"
    question = "./dataset/process/SPIDER-TEST_SQL_9-SHOT_EUCDISQUESTIONMASK_QA-EXAMPLE_CTX-200_ANS-4096_test/"
    start_index = 0
    end_index = 4
    batch_size =  4

In [3]:
args.question

'./dataset/process/SPIDER-TEST_SQL_9-SHOT_EUCDISQUESTIONMASK_QA-EXAMPLE_CTX-200_ANS-4096/'

In [4]:
questions_json = json.load(open(os.path.join(args.question, QUESTION_FILE), "r"))
questions = [_["prompt"] for _ in questions_json["questions"]]
db_ids = [_["db_id"] for _ in questions_json["questions"]]

question_loader = DataLoader(questions, batch_size=args.batch_size, shuffle=False, drop_last=False)
out_file = f"{args.question}/RESULTS_MODEL-{args.model}.txt"

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
model_path = LLM.OPEN_SOURCE_2_PATH[args.model]
tokenizer = AutoTokenizer.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
model.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-35): 36 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=True)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=True)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=True)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (l

In [6]:
questions = ['/* Some SQL examples are provided based on similar problems: */\n/* Answer the following: How many aircrafts do we have? */\nSELECT count(*) FROM Aircraft\n\n/* Answer the following: How many employees do we have? */\nSELECT count(*) FROM Employee\n\n/* Answer the following: How many flights do we have? */\nSELECT count(*) FROM Flight\n\n/* Answer the following: How many accounts do we have? */\nSELECT count(*) FROM Accounts\n\n/* Answer the following: How many customers do we have? */\nSELECT count(*) FROM Customers\n\n/* Answer the following: How many tracks do we have? */\nSELECT count(*) FROM track\n\n/* Answer the following: How many transactions do we have? */\nSELECT count(*) FROM Financial_transactions\n\n/* Answer the following: How many invoices do we have? */\nSELECT count(*) FROM Invoices\n\n/* Answer the following: How many artists do we have? */\nSELECT count(*) FROM artist\n\n/* Given the following database schema: */\nCREATE TABLE "stadium" (\n"Stadium_ID" int,\n"Location" text,\n"Name" text,\n"Capacity" int,\n"Highest" int,\n"Lowest" int,\n"Average" int,\nPRIMARY KEY ("Stadium_ID")\n)\n\nCREATE TABLE "singer" (\n"Singer_ID" int,\n"Name" text,\n"Country" text,\n"Song_Name" text,\n"Song_release_year" text,\n"Age" int,\n"Is_male" bool,\nPRIMARY KEY ("Singer_ID")\n)\n\nCREATE TABLE "concert" (\n"concert_ID" int,\n"concert_Name" text,\n"Theme" text,\n"Stadium_ID" text,\n"Year" text,\nPRIMARY KEY ("concert_ID"),\nFOREIGN KEY ("Stadium_ID") REFERENCES "stadium"("Stadium_ID")\n)\n\nCREATE TABLE "singer_in_concert" (\n"concert_ID" int,\n"Singer_ID" text,\nPRIMARY KEY ("concert_ID","Singer_ID"),\nFOREIGN KEY ("concert_ID") REFERENCES "concert"("concert_ID"),\nFOREIGN KEY ("Singer_ID") REFERENCES "singer"("Singer_ID")\n)\n\n/* Answer the following: How many singers do we have? */\nSELECT ', 
    '/* Some SQL examples are provided based on similar problems: */\n/* Answer the following: What is the total number of students? */\nSELECT count(*) FROM Student\n\n/* Answer the following: What is the total number of companies? */\nSELECT count(*) FROM company\n\n/* Answer the following: What is the total number of campuses? */\nSELECT count(*) FROM campuses\n\n/* Answer the following: What is the total number of games played? */\nSELECT sum(gamesplayed) FROM Sportsinfo\n\n/* Answer the following: What is the total count of enzymes? */\nSELECT count(*) FROM enzyme\n\n/* Answer the following: What is the number of employees? */\nSELECT count(*) FROM Employee\n\n/* Answer the following: What is the number of flights? */\nSELECT count(*) FROM Flight\n\n/* Answer the following: What is the number of technicians? */\nSELECT count(*) FROM technician\n\n/* Answer the following: What is the number of ships? */\nSELECT count(*) FROM ship\n\n/* Given the following database schema: */\nCREATE TABLE "stadium" (\n"Stadium_ID" int,\n"Location" text,\n"Name" text,\n"Capacity" int,\n"Highest" int,\n"Lowest" int,\n"Average" int,\nPRIMARY KEY ("Stadium_ID")\n)\n\nCREATE TABLE "singer" (\n"Singer_ID" int,\n"Name" text,\n"Country" text,\n"Song_Name" text,\n"Song_release_year" text,\n"Age" int,\n"Is_male" bool,\nPRIMARY KEY ("Singer_ID")\n)\n\nCREATE TABLE "concert" (\n"concert_ID" int,\n"concert_Name" text,\n"Theme" text,\n"Stadium_ID" text,\n"Year" text,\nPRIMARY KEY ("concert_ID"),\nFOREIGN KEY ("Stadium_ID") REFERENCES "stadium"("Stadium_ID")\n)\n\nCREATE TABLE "singer_in_concert" (\n"concert_ID" int,\n"Singer_ID" text,\nPRIMARY KEY ("concert_ID","Singer_ID"),\nFOREIGN KEY ("concert_ID") REFERENCES "concert"("concert_ID"),\nFOREIGN KEY ("Singer_ID") REFERENCES "singer"("Singer_ID")\n)\n\n/* Answer the following: What is the total number of singers? */\nSELECT ', 
    '/* Some SQL examples are provided based on similar problems: */\n/* Answer the following: Show the working years of managers in descending order of their level. */\nSELECT Working_year_starts FROM manager ORDER BY LEVEL DESC\n\n/* Answer the following: Find the name of the students and their department names sorted by their total credits in ascending order. */\nSELECT name ,  dept_name FROM student ORDER BY tot_cred\n\n/* Answer the following: Show the names of members in ascending order of their rank in rounds. */\nSELECT T1.Name FROM member AS T1 JOIN round AS T2 ON T1.Member_ID  =  T2.Member_ID ORDER BY Rank_in_Round ASC\n\n/* Answer the following: Find the name, headquarter and revenue of all manufacturers sorted by their revenue in the descending order. */\nSELECT name ,  headquarter ,  revenue FROM manufacturers ORDER BY revenue DESC\n\n/* Answer the following: For each party, find its location and the name of its host. Sort the result in ascending order of the age of the host. */\nSELECT T3.Location ,  T2.Name FROM party_host AS T1 JOIN HOST AS T2 ON T1.Host_ID  =  T2.Host_ID JOIN party AS T3 ON T1.Party_ID  =  T3.Party_ID ORDER BY T2.Age\n\n/* Answer the following: List all the cities in a decreasing order of each city\'s stations\' highest latitude. */\nSELECT city FROM station GROUP BY city ORDER BY max(lat) DESC\n\n/* Answer the following: Show names of actors in descending order of the year their musical is awarded. */\nSELECT T1.Name FROM actor AS T1 JOIN musical AS T2 ON T1.Musical_ID  =  T2.Musical_ID ORDER BY T2.Year DESC\n\n/* Answer the following: Show the name and service for all trains in order by time. */\nSELECT name ,  service FROM train ORDER BY TIME\n\n/* Answer the following: Show all movie titles, years, and directors, ordered by budget. */\nSELECT title ,  YEAR ,  director FROM movie ORDER BY budget_million\n\n/* Given the following database schema: */\nCREATE TABLE "stadium" (\n"Stadium_ID" int,\n"Location" text,\n"Name" text,\n"Capacity" int,\n"Highest" int,\n"Lowest" int,\n"Average" int,\nPRIMARY KEY ("Stadium_ID")\n)\n\nCREATE TABLE "singer" (\n"Singer_ID" int,\n"Name" text,\n"Country" text,\n"Song_Name" text,\n"Song_release_year" text,\n"Age" int,\n"Is_male" bool,\nPRIMARY KEY ("Singer_ID")\n)\n\nCREATE TABLE "concert" (\n"concert_ID" int,\n"concert_Name" text,\n"Theme" text,\n"Stadium_ID" text,\n"Year" text,\nPRIMARY KEY ("concert_ID"),\nFOREIGN KEY ("Stadium_ID") REFERENCES "stadium"("Stadium_ID")\n)\n\nCREATE TABLE "singer_in_concert" (\n"concert_ID" int,\n"Singer_ID" text,\nPRIMARY KEY ("concert_ID","Singer_ID"),\nFOREIGN KEY ("concert_ID") REFERENCES "concert"("concert_ID"),\nFOREIGN KEY ("Singer_ID") REFERENCES "singer"("Singer_ID")\n)\n\n/* Answer the following: Show name, country, age for all singers ordered by age from the oldest to the youngest. */\nSELECT ', 
    '/* Some SQL examples are provided based on similar problems: */\n/* Answer the following: What are the years, titles, and publishers for all books, ordered by year descending? */\nSELECT YEAR ,  book_title ,  publisher FROM book_club ORDER BY YEAR DESC\n\n/* Answer the following: What are the card numbers, names, and hometowns of every member ordered by descending level? */\nSELECT card_number ,  name ,  hometown FROM member ORDER BY LEVEL DESC\n\n/* Answer the following: What are the total points for all gymnasts, ordered by total points descending? */\nSELECT Total_Points FROM gymnast ORDER BY Total_Points DESC\n\n/* Answer the following: What are the themes and years for exhibitions, sorted by ticket price descending? */\nSELECT theme ,  YEAR FROM exhibition ORDER BY ticket_price DESC\n\n/* Answer the following: What are the names of enzymes in descending order? */\nSELECT name FROM enzyme ORDER BY name DESC\n\n/* Answer the following: What are the lifespans of representatives in descending order of vote percent? */\nSELECT T2.Lifespan FROM election AS T1 JOIN representative AS T2 ON T1.Representative_ID  =  T2.Representative_ID ORDER BY Vote_Percent DESC\n\n/* Answer the following: What are the weights of entrepreneurs in descending order of money requested? */\nSELECT T2.Weight FROM entrepreneur AS T1 JOIN people AS T2 ON T1.People_ID  =  T2.People_ID ORDER BY T1.Money_Requested DESC\n\n/* Answer the following: What are the characters of actors in descending order of age? */\nSELECT Character FROM actor ORDER BY age DESC\n\n/* Answer the following: What are the names of representatives in descending order of votes? */\nSELECT T2.Name FROM election AS T1 JOIN representative AS T2 ON T1.Representative_ID  =  T2.Representative_ID ORDER BY votes DESC\n\n/* Given the following database schema: */\nCREATE TABLE "stadium" (\n"Stadium_ID" int,\n"Location" text,\n"Name" text,\n"Capacity" int,\n"Highest" int,\n"Lowest" int,\n"Average" int,\nPRIMARY KEY ("Stadium_ID")\n)\n\nCREATE TABLE "singer" (\n"Singer_ID" int,\n"Name" text,\n"Country" text,\n"Song_Name" text,\n"Song_release_year" text,\n"Age" int,\n"Is_male" bool,\nPRIMARY KEY ("Singer_ID")\n)\n\nCREATE TABLE "concert" (\n"concert_ID" int,\n"concert_Name" text,\n"Theme" text,\n"Stadium_ID" text,\n"Year" text,\nPRIMARY KEY ("concert_ID"),\nFOREIGN KEY ("Stadium_ID") REFERENCES "stadium"("Stadium_ID")\n)\n\nCREATE TABLE "singer_in_concert" (\n"concert_ID" int,\n"Singer_ID" text,\nPRIMARY KEY ("concert_ID","Singer_ID"),\nFOREIGN KEY ("concert_ID") REFERENCES "concert"("concert_ID"),\nFOREIGN KEY ("Singer_ID") REFERENCES "singer"("Singer_ID")\n)\n\n/* Answer the following: What are the names, countries, and ages for every singer in descending order of age? */\nSELECT ']

In [6]:
# with open("./results/re_text.txt", "w") as f1, open("./results/original_text.txt", "w") as f2:
#     chat = get_template(questions)
#     print(chat)
#     chat = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt", padding=True, truncation=True)
#     chat = chat.cuda()
#     inputs = tokenizer.batch_decode(chat, skip_special_tokens=True)
#     outputs = model.generate(chat, max_new_tokens=2048)
#     outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#     for i, output in enumerate(outputs):
#         f1.write(output)
with open(out_file, "w") as f1, open("./results/original_text.txt", "w") as f2:
    for i, batch in enumerate(tqdm(question_loader)):
        chat = get_template(batch)
        chat = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt", padding=True, truncation=True)
        chat = chat.cuda()
        outputs = model.generate(chat, max_new_tokens=2048)

        outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        for i, output in enumerate(outputs):
            try:
                ans = get_answer(output)
                f1.write(ans + '\n')
            except Exception as e:
                f1.write('SELECT \n')
            finally:
                f2.write(output + '\n')

 45%|███████████████████████████████████████████████████████████▊                                                                          | 29/65 [1:32:54<1:55:19, 192.22s/it]


KeyboardInterrupt: 

In [ ]:
"jello"